In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as smi
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from statistics import mean
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, SimpleRNN, GRU, Dropout, Conv1D, MaxPooling1D, Flatten, RepeatVector
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error, mean_absolute_error
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
from statsmodels.tsa.seasonal import seasonal_decompose
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from hyperopt import hp, Trials, fmin, tpe
import joblib
import warnings
warnings.simplefilter('ignore')

In [2]:
df=pd.read_csv(r'Niftymultifatorinput_with sensex and bank nifty data.csv')
df.head()

,Date,NIFTY,Crude Oil,India VIX,INRUSD,Gold,S & P 500,Sensex,NIFTY BANK
0,1/2/2009,3046.750000,46.340000,40.67,48.24,878.8,931.80,9958.219727,5195.10
1,1/5/2009,3121.449951,48.810001,38.60,48.23,857.2,927.45,10275.599610,5359.40
2,1/6/2009,3112.800049,48.580002,38.73,48.51,865.4,934.70,10335.929690,5395.30
3,1/7/2009,2920.399902,42.630001,44.36,48.55,841.1,906.65,9586.879883,4984.55
4,1/9/2009,2873.000000,41.700001,47.82,48.47,854.3,890.35,9406.469727,4906.70


In [3]:
#feature_columns = ['Sensex', 'Crude Oil', 'India VIX', 'Gold', 'RSI', 'MACD', 'INRUSD', 'S & P 500']
feature_columns = ['Sensex', 'INRUSD', 'S & P 500']
target_column = ['Sensex']
features = df[feature_columns]
target = df[target_column].shift(-1)
target=target.dropna()

features =df[feature_columns].head(-1)
features.tail(5)

,Sensex,INRUSD,S & P 500
3668,70865.10156,83.27,4746.75
3669,71106.96094,83.13,4754.63
3670,71336.80000,83.17,4774.75
3671,72038.43000,83.18,4781.58
3672,72410.38000,83.27,4783.35


In [4]:
target.tail(5)

,Sensex
3668,71106.96094
3669,71336.80000
3670,72038.43000
3671,72410.38000
3672,72240.26000


In [5]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)
scaled_target = scaler.fit_transform(target.values.reshape(-1, 1))

n_steps = 5  # Number of timesteps to look back
X, y = [], []

for i in range(n_steps, len(scaled_features)):
    X.append(scaled_features[i-n_steps:i])
    y.append(scaled_target[i])

X = np.array(X)
y = np.array(y)

split=int(0.8*len(X))
X_train, y_train= X[:split], y[:split]
X_test, y_test= X[split:], y[split:]

In [6]:
#load saved LSTM 1 layer best model
model_lstm = load_model('model_ET_sp-usdinr_lstm1_0.9752.keras')

#Test the model with Outsample data for model Validation
model_lstm.predict(X_test)
predictions_lstm = model_lstm.predict(X_test)
predictions_LSTM = scaler.inverse_transform(predictions_lstm)
y_test_actual_LSTM = scaler.inverse_transform(y_test.reshape(-1, 1))

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_LSTM, predictions_LSTM)
print(f"R2: {r2:.4f}")

mape=mean_absolute_percentage_error(y_test_actual_LSTM, predictions_LSTM)
print(f"MAPE: {mape:.4f}")

mse=mean_squared_error(y_test_actual_LSTM, predictions_LSTM)
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

mse_normalizeddata=mean_squared_error(y_test, predictions_lstm)
rmse_normalizeddata=np.sqrt(mse_normalizeddata)
print(f"RMSE_normalizeddata: {rmse_normalizeddata:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_lstm)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
R2: 0.9745
MAPE: 0.0114
RMSE: 846.2954
RMSE_normalizeddata: 0.0132
MAE_normalizeddata: 0.0102


In [7]:
#load saved GRU 1 layer best model
model_gru = load_model('model_ET_sp-usdinr_gru1_0.9775.keras')

#Test the model with Outsample data for model Validation
model_gru.predict(X_test)
predictions_gru = model_gru.predict(X_test)
predictions_GRU = scaler.inverse_transform(predictions_gru)
y_test_actual_GRU = scaler.inverse_transform(y_test.reshape(-1, 1))

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_GRU, predictions_GRU)
print(f"R2: {r2:.4f}")
mape=mean_absolute_percentage_error(y_test_actual_GRU, predictions_GRU)
print(f"MAPE: {mape:.4f}")
mse=mean_squared_error(y_test_actual_GRU, predictions_GRU)
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

mse_normalizeddata=mean_squared_error(y_test, predictions_gru)
rmse_normalizeddata=np.sqrt(mse_normalizeddata)
print(f"RMSE_normalizeddata: {rmse_normalizeddata:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_gru)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
R2: 0.9772
MAPE: 0.0108
RMSE: 801.0177
RMSE_normalizeddata: 0.0125
MAE_normalizeddata: 0.0096


In [8]:
#load saved GRU-LSTM 1 layer best model
model_grulstm = load_model('model_ET_sp-usdinr_grulstm1_0.9797.keras')

#Test the model with Outsample data for model Validation
model_grulstm.predict(X_test)
predictions_grulstm = model_grulstm.predict(X_test)
predictions_GRULSTM = scaler.inverse_transform(predictions_grulstm)
y_test_actual_GRULSTM = scaler.inverse_transform(y_test.reshape(-1, 1))


# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"R2: {r2:.4f}")

mape=mean_absolute_percentage_error(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"MAPE: {mape:.4f}")

mse=mean_squared_error(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"MSE: {mse:.4f}")

rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_grulstm)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

mse_n=mean_squared_error(y_test, predictions_grulstm)
rmse_n=np.sqrt(mse_n)
print(f"RMSE of Normalized Data: {rmse_n:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
R2: 0.9791
MAPE: 0.0104
MSE: 588871.6641
RMSE: 767.3797
MAE_normalizeddata: 0.0093
RMSE of Normalized Data: 0.0119


In [9]:
#load saved LSTM-GRU 1 layer best model
model_lstmgru = load_model('model_ET_sp-usdinr_lstmgru1_0.9697.keras')
model_lstmgru.predict(X_test)

#Test the model with Outsample data for model Validation
predictions_lstmgru = model_lstmgru.predict(X_test)
predictions_LSTMGRU = scaler.inverse_transform(predictions_lstmgru)
y_test_actual_LSTMGRU = scaler.inverse_transform(y_test.reshape(-1, 1))

dates_test=df['Date'].iloc[split + n_steps:]

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"R2: {r2:.4f}")

mape=mean_absolute_percentage_error(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"MAPE: {mape:.4f}")

mse=mean_squared_error(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"MSE: {mse:.4f}")
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

mae=mean_absolute_error(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"MAE: {mae:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_lstmgru)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

mse_n=mean_squared_error(y_test, predictions_lstmgru)
rmse_n=np.sqrt(mse_n)
print(f"RMSE of Normalized Data: {rmse_n:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
R2: 0.9692
MAPE: 0.0128
MSE: 867040.0203
RMSE: 931.1498
MAE: 734.3166
MAE_normalizeddata: 0.0114
RMSE of Normalized Data: 0.0145


In [10]:
#load saved LSTM 2 layer best model
model_lstm = load_model('model_ET_sp-usdinr_lstm2_0.9725.keras')

#Test the model with Outsample data for model Validation
model_lstm.predict(X_test)
predictions_lstm = model_lstm.predict(X_test)
predictions_LSTM = scaler.inverse_transform(predictions_lstm)
y_test_actual_LSTM = scaler.inverse_transform(y_test.reshape(-1, 1))

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_LSTM, predictions_LSTM)
print(f"R2: {r2:.4f}")

mape=mean_absolute_percentage_error(y_test_actual_LSTM, predictions_LSTM)
print(f"MAPE: {mape:.4f}")

mse=mean_squared_error(y_test_actual_LSTM, predictions_LSTM)
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

mse_normalizeddata=mean_squared_error(y_test, predictions_lstm)
rmse_normalizeddata=np.sqrt(mse_normalizeddata)
print(f"RMSE_normalizeddata: {rmse_normalizeddata:.4f}")

mae=mean_absolute_error(y_test_actual_LSTM, predictions_LSTM)
print(f"MAE: {mae:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_lstm)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
R2: 0.9712
MAPE: 0.0125
RMSE: 899.4223
RMSE_normalizeddata: 0.0140
MAE: 728.8469
MAE_normalizeddata: 0.0113


In [11]:
#load saved GRU 2 layer best model
model_gru = load_model('model_ET_sp-usdinr_gru2_0.9626.keras')

#Test the model with Outsample data for model Validation
model_gru.predict(X_test)
predictions_gru = model_gru.predict(X_test)
predictions_GRU = scaler.inverse_transform(predictions_gru)
y_test_actual_GRU = scaler.inverse_transform(y_test.reshape(-1, 1))

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_GRU, predictions_GRU)
print(f"R2: {r2:.4f}")
mape=mean_absolute_percentage_error(y_test_actual_GRU, predictions_GRU)
print(f"MAPE: {mape:.4f}")
mse=mean_squared_error(y_test_actual_GRU, predictions_GRU)
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")
mse_normalizeddata=mean_squared_error(y_test, predictions_gru)
rmse_normalizeddata=np.sqrt(mse_normalizeddata)
print(f"RMSE_normalizeddata: {rmse_normalizeddata:.4f}")
mae=mean_absolute_error(y_test_actual_GRU, predictions_GRU)
print(f"MAE: {mae:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_gru)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
R2: 0.9616
MAPE: 0.0141
RMSE: 1038.8149
RMSE_normalizeddata: 0.0162
MAE: 813.1797
MAE_normalizeddata: 0.0127


In [12]:
#load saved GRU-LSTM 2 layer best model
model_grulstm = load_model('model_ET_sp-usdinr_grulstm2_0.9594.keras')

#Test the model with Outsample data for model Validation
model_grulstm.predict(X_test)
predictions_grulstm = model_grulstm.predict(X_test)
predictions_GRULSTM = scaler.inverse_transform(predictions_grulstm)
y_test_actual_GRULSTM = scaler.inverse_transform(y_test.reshape(-1, 1))

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"R2: {r2:.4f}")

mape=mean_absolute_percentage_error(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"MAPE: {mape:.4f}")

mse=mean_squared_error(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"MSE: {mse:.4f}")
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

mae=mean_absolute_error(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"MAE: {mae:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_grulstm)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

mse_n=mean_squared_error(y_test, predictions_grulstm)
rmse_n=np.sqrt(mse_n)
print(f"RMSE of Normalized Data: {rmse_n:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step
R2: 0.9573
MAPE: 0.0152
MSE: 1200186.8706
RMSE: 1095.5304
MAE: 875.9110
MAE_normalizeddata: 0.0136
RMSE of Normalized Data: 0.0171


In [13]:
#load saved LSTM-GRU 2 layer best model
model_lstmgru = load_model('model_ET_sp-usdinr_lstmgru2_0.9512.keras')

#Test the model with Outsample data for model Validation
model_lstmgru.predict(X_test)
predictions_lstmgru = model_lstmgru.predict(X_test)
predictions_LSTMGRU = scaler.inverse_transform(predictions_lstmgru)
y_test_actual_LSTMGRU = scaler.inverse_transform(y_test.reshape(-1, 1))

dates_test=df['Date'].iloc[split + n_steps:]

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"R2: {r2:.4f}")

mape=mean_absolute_percentage_error(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"MAPE: {mape:.4f}")

mse=mean_squared_error(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"MSE: {mse:.4f}")
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

mae=mean_absolute_error(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"MAE: {mae:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_lstmgru)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

mse_n=mean_squared_error(y_test, predictions_lstmgru)
rmse_n=np.sqrt(mse_n)
print(f"RMSE of Normalized Data: {rmse_n:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
R2: 0.9489
MAPE: 0.0170
MSE: 1436810.2753
RMSE: 1198.6702
MAE: 981.8108
MAE_normalizeddata: 0.0153
RMSE of Normalized Data: 0.0187


In [14]:
#load saved LSTM 3 layer best model
model_lstm = load_model('model_ET_sp-usdinr_lstm3_0.9501.keras')

#Test the model with Outsample data for model Validation
model_lstm.predict(X_test)
predictions_lstm = model_lstm.predict(X_test)
predictions_LSTM = scaler.inverse_transform(predictions_lstm)
y_test_actual_LSTM = scaler.inverse_transform(y_test.reshape(-1, 1))

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_LSTM, predictions_LSTM)
print(f"R2: {r2:.4f}")

mape=mean_absolute_percentage_error(y_test_actual_LSTM, predictions_LSTM)
print(f"MAPE: {mape:.4f}")

mse=mean_squared_error(y_test_actual_LSTM, predictions_LSTM)
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

mse_normalizeddata=mean_squared_error(y_test, predictions_lstm)
rmse_normalizeddata=np.sqrt(mse_normalizeddata)
print(f"RMSE_normalizeddata: {rmse_normalizeddata:.4f}")

mae=mean_absolute_error(y_test_actual_LSTM, predictions_LSTM)
print(f"MAE: {mae:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_lstm)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
R2: 0.9521
MAPE: 0.0156
RMSE: 1160.5203
RMSE_normalizeddata: 0.0181
MAE: 901.8407
MAE_normalizeddata: 0.0140


In [15]:
#load saved GRU 3 layer best model
model_gru = load_model('model_ET_sp-usdinr_gru3_0.9745.keras')

#Test the model with Outsample data for model Validation
model_gru.predict(X_test)
predictions_gru = model_gru.predict(X_test)
predictions_GRU = scaler.inverse_transform(predictions_gru)
y_test_actual_GRU = scaler.inverse_transform(y_test.reshape(-1, 1))

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_GRU, predictions_GRU)
print(f"R2: {r2:.4f}")
mape=mean_absolute_percentage_error(y_test_actual_GRU, predictions_GRU)
print(f"MAPE: {mape:.4f}")
mse=mean_squared_error(y_test_actual_GRU, predictions_GRU)
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")
mse_normalizeddata=mean_squared_error(y_test, predictions_gru)
rmse_normalizeddata=np.sqrt(mse_normalizeddata)
print(f"RMSE_normalizeddata: {rmse_normalizeddata:.4f}")
mae=mean_absolute_error(y_test_actual_GRU, predictions_GRU)
print(f"MAE: {mae:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_gru)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
R2: 0.9732
MAPE: 0.0117
RMSE: 867.5087
RMSE_normalizeddata: 0.0135
MAE: 675.3720
MAE_normalizeddata: 0.0105


In [19]:
#load saved GRU-LSTM 3 layer best model
model_grulstm = load_model('model_ET_sp-usdinr_grulstm3_0.9396.keras')

#Test the model with Outsample data for model Validation
model_grulstm.predict(X_test)
predictions_grulstm = model_grulstm.predict(X_test)
predictions_GRULSTM = scaler.inverse_transform(predictions_grulstm)
y_test_actual_GRULSTM = scaler.inverse_transform(y_test.reshape(-1, 1))
dates_test=df['Date'].iloc[split + n_steps:]

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"R2: {r2:.4f}")
mape=mean_absolute_percentage_error(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"MAPE: {mape:.4f}")
mse=mean_squared_error(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"MSE: {mse:.4f}")
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")
mae=mean_absolute_error(y_test_actual_GRULSTM, predictions_GRULSTM)
print(f"MAE: {mae:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_grulstm)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

mse_normalizeddata=mean_squared_error(y_test, predictions_grulstm)
rmse_normalizeddata=np.sqrt(mse_normalizeddata)
print(f"RMSE of Normalized Data: {rmse_normalizeddata:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
R2: 0.9398
MAPE: 0.0179
MSE: 1692713.9228
RMSE: 1301.0434
MAE: 1047.3056
MAE_normalizeddata: 0.0163
RMSE of Normalized Data: 0.0202


In [18]:
#load saved LSTM-GRU 3 layer best model
model_lstmgru = load_model('model_ET_sp-usdinr_lstmgru3_0.9420.keras')

#Test the model with Outsample data for model Validation
model_lstmgru.predict(X_test)
predictions_lstmgru = model_lstmgru.predict(X_test)
predictions_LSTMGRU = scaler.inverse_transform(predictions_lstmgru)
y_test_actual_LSTMGRU = scaler.inverse_transform(y_test.reshape(-1, 1))
dates_test=df['Date'].iloc[split + n_steps:]

# Model's Accuracy Calculation 
r2=r2_score(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"R2: {r2:.4f}")

mape=mean_absolute_percentage_error(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"MAPE: {mape:.4f}")

mse=mean_squared_error(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"MSE: {mse:.4f}")
rmse=np.sqrt(mse)
print(f"RMSE: {rmse:.4f}")

mae=mean_absolute_error(y_test_actual_LSTMGRU, predictions_LSTMGRU)
print(f"MAE: {mae:.4f}")

mae_normalizeddata=mean_absolute_error(y_test, predictions_lstmgru)
print(f"MAE_normalizeddata: {mae_normalizeddata:.4f}")

mse_normalizeddata=mean_squared_error(y_test, predictions_lstmgru)
rmse_normalizeddata=np.sqrt(mse_normalizeddata)
print(f"RMSE of Normalized Data: {rmse_normalizeddata:.4f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
R2: 0.9391
MAPE: 0.0181
MSE: 1712132.0939
RMSE: 1308.4847
MAE: 1057.3045
MAE_normalizeddata: 0.0165
RMSE of Normalized Data: 0.0204
